In [16]:
import pandas as pd
import sqlite3

# Check what's actually in the database vs CSV
def debug_foreign_key_issue():
    # Read the CSV files
    products_df = pd.read_csv('../data/olist_products_dataset.csv')
    categories_df = pd.read_csv('../data/product_category_name_translation.csv')
    
    print("=== FOREIGN KEY DEBUG ===")
    
    # Check categories loaded in database
    conn = sqlite3.connect('../olist_dashboard.db')
    
    db_categories = pd.read_sql("SELECT product_category_name FROM dim_product_categories", conn)
    print(f"Categories in database: {len(db_categories)}")
    print(f"Categories in CSV: {len(categories_df)}")
    
    # Check unique categories in products CSV
    product_categories = products_df['product_category_name'].dropna().unique()
    csv_categories = categories_df['product_category_name'].unique()
    
    print(f"Unique categories in products CSV: {len(product_categories)}")
    print(f"Categories in translation CSV: {len(csv_categories)}")
    
    # Find missing categories
    missing_categories = set(product_categories) - set(csv_categories)
    print(f"\nMissing categories (in products but not in translation): {len(missing_categories)}")
    if missing_categories:
        print("Missing categories:", list(missing_categories)[:10])  # Show first 10
    
    # Check NULL values
    null_categories = products_df['product_category_name'].isnull().sum()
    print(f"\nNULL categories in products: {null_categories}")
    
    # Check what's actually in the database
    db_category_list = db_categories['product_category_name'].tolist()
    print(f"\nFirst 10 categories in database: {db_category_list[:10]}")
    
    conn.close()
    
    return missing_categories

if __name__ == "__main__":
    missing = debug_foreign_key_issue()

=== FOREIGN KEY DEBUG ===
Categories in database: 74
Categories in CSV: 71
Unique categories in products CSV: 73
Categories in translation CSV: 71

Missing categories (in products but not in translation): 2
Missing categories: ['pc_gamer', 'portateis_cozinha_e_preparadores_de_alimentos']

NULL categories in products: 610

First 10 categories in database: ['agro_industria_e_comercio', 'alimentos', 'alimentos_bebidas', 'artes', 'artes_e_artesanato', 'artigos_de_festas', 'artigos_de_natal', 'audio', 'automotivo', 'bebes']


In [17]:
import pandas as pd

def analyze_null_products():
    """Analyze products with NULL categories to see if they're worth keeping"""
    
    df = pd.read_csv('../data/olist_products_dataset.csv')
    
    print("=== NULL CATEGORY PRODUCTS ANALYSIS ===")
    
    # Separate NULL and non-NULL products
    null_products = df[df['product_category_name'].isnull()]
    valid_products = df[df['product_category_name'].notnull()]
    
    print(f"Products with NULL categories: {len(null_products)}")
    print(f"Products with valid categories: {len(valid_products)}")
    
    # Check if NULL products have other useful data
    print(f"\n=== NULL PRODUCTS DATA QUALITY ===")
    
    # Check non-category columns for completeness
    null_cols_analysis = {}
    for col in null_products.columns:
        if col != 'product_category_name':
            non_null_count = null_products[col].notnull().sum()
            null_cols_analysis[col] = {
                'non_null': non_null_count,
                'null': len(null_products) - non_null_count,
                'completeness_pct': (non_null_count / len(null_products)) * 100
            }
    
    print("Data completeness for NULL category products:")
    for col, stats in null_cols_analysis.items():
        print(f"  {col}: {stats['completeness_pct']:.1f}% complete ({stats['non_null']}/{len(null_products)})")
    
    # Sample NULL products to see what they look like
    print(f"\n=== SAMPLE NULL CATEGORY PRODUCTS ===")
    print(null_products[['product_id', 'product_name_lenght', 'product_weight_g', 
                        'product_length_cm', 'product_height_cm', 'product_width_cm']].head(10))
    
    # Check if NULL products are referenced in order_items (i.e., actually sold)
    order_items = pd.read_csv('../data/olist_order_items_dataset.csv')
    null_product_ids = set(null_products['product_id'])
    sold_null_products = order_items[order_items['product_id'].isin(null_product_ids)]
    
    print(f"\n=== NULL PRODUCTS IN ORDERS ===")
    print(f"NULL category products that were actually sold: {len(sold_null_products)}")
    print(f"Unique NULL products sold: {sold_null_products['product_id'].nunique()}")
    
    if len(sold_null_products) > 0:
        total_revenue = sold_null_products['price'].sum()
        print(f"Total revenue from NULL category products: ${total_revenue:,.2f}")
    
    return null_products, sold_null_products

def analyze_missing_categories():
    """Analyze the 2 missing categories"""
    
    df = pd.read_csv('../data/olist_products_dataset.csv')
    
    missing_cats = ['pc_gamer', 'portateis_cozinha_e_preparadores_de_alimentos']
    
    print(f"\n=== MISSING CATEGORIES ANALYSIS ===")
    
    for cat in missing_cats:
        cat_products = df[df['product_category_name'] == cat]
        print(f"\nCategory: {cat}")
        print(f"Number of products: {len(cat_products)}")
        
        if len(cat_products) > 0:
            # Check if these products are sold
            order_items = pd.read_csv('../data/olist_order_items_dataset.csv')
            cat_product_ids = set(cat_products['product_id'])
            sold_cat_products = order_items[order_items['product_id'].isin(cat_product_ids)]
            
            print(f"Products sold: {sold_cat_products['product_id'].nunique()}")
            if len(sold_cat_products) > 0:
                revenue = sold_cat_products['price'].sum()
                print(f"Revenue: ${revenue:,.2f}")
                avg_price = sold_cat_products['price'].mean()
                print(f"Average price: ${avg_price:.2f}")

null_products, sold_nulls = analyze_null_products()
analyze_missing_categories()

=== NULL CATEGORY PRODUCTS ANALYSIS ===
Products with NULL categories: 610
Products with valid categories: 32341

=== NULL PRODUCTS DATA QUALITY ===
Data completeness for NULL category products:
  product_id: 100.0% complete (610/610)
  product_name_lenght: 0.0% complete (0/610)
  product_description_lenght: 0.0% complete (0/610)
  product_photos_qty: 0.0% complete (0/610)
  product_weight_g: 99.8% complete (609/610)
  product_length_cm: 99.8% complete (609/610)
  product_height_cm: 99.8% complete (609/610)
  product_width_cm: 99.8% complete (609/610)

=== SAMPLE NULL CATEGORY PRODUCTS ===
                           product_id  product_name_lenght  product_weight_g  \
105  a41e356c76fab66334f36de622ecbd3a                  NaN             650.0   
128  d8dee61c2034d6d075997acef1870e9b                  NaN             300.0   
145  56139431d72cd51f19eb9f7dae4d1617                  NaN             200.0   
154  46b48281eb6d663ced748f324108c733                  NaN           18500.0   
197

NULL Category Products:

610 products with NULL categories that were ALL sold (100% sell-through)
$179,535 in revenue - significant business impact
Physical dimensions are 99.8% complete - good quality data
Only missing description/name length and photo count (not critical for analysis)


ADD NULL CATEGORY IN SCHEMA AND ADD MISSING PRODUCT CATEGORIES

In [18]:
import pandas as pd

def check_order_statuses():
    """Check what order statuses exist in the data"""
    
    df = pd.read_csv('../data/olist_orders_dataset2.csv')
    
    print("=== ORDER STATUS ANALYSIS ===")
    
    # Get all unique statuses
    all_statuses = df['order_status'].unique()
    print(f"Unique statuses in data: {len(all_statuses)}")
    print("Statuses found:", sorted(all_statuses))
    
    # Schema allowed statuses
    allowed_statuses = ['delivered', 'shipped', 'processing', 'unavailable', 'canceled', 'approved', 'invoiced']
    print(f"\nAllowed in schema: {allowed_statuses}")
    
    # Find invalid statuses
    invalid_statuses = set(all_statuses) - set(allowed_statuses)
    print(f"\nInvalid statuses: {sorted(invalid_statuses)}")
    
    # Count orders for each status
    status_counts = df['order_status'].value_counts()
    print(f"\nStatus counts:")
    for status in sorted(all_statuses):
        count = status_counts[status]
        valid = "✓" if status in allowed_statuses else "✗"
        print(f"  {valid} {status}: {count:,} orders")
    
    return invalid_statuses

if __name__ == "__main__":
    invalid = check_order_statuses()

=== ORDER STATUS ANALYSIS ===
Unique statuses in data: 8
Statuses found: ['approved', 'canceled', 'created', 'delivered', 'invoiced', 'processing', 'shipped', 'unavailable']

Allowed in schema: ['delivered', 'shipped', 'processing', 'unavailable', 'canceled', 'approved', 'invoiced']

Invalid statuses: ['created']

Status counts:
  ✓ approved: 2 orders
  ✓ canceled: 625 orders
  ✗ created: 5 orders
  ✓ delivered: 96,478 orders
  ✓ invoiced: 314 orders
  ✓ processing: 301 orders
  ✓ shipped: 1,107 orders
  ✓ unavailable: 609 orders


Now you have duplicate review_id values in your reviews data. This means your CSV file has the same review ID appearing multiple times, which violates the PRIMARY KEY constraint. -- CHECK DUPLICATES

In [19]:
import pandas as pd

def check_review_duplicates():
    """Check for duplicate review IDs in the data"""
    
    df = pd.read_csv('../data/olist_order_reviews_dataset.csv')
    
    print("=== REVIEW DUPLICATES ANALYSIS ===")
    
    total_reviews = len(df)
    unique_review_ids = df['review_id'].nunique()
    
    print(f"Total reviews: {total_reviews:,}")
    print(f"Unique review IDs: {unique_review_ids:,}")
    print(f"Duplicate reviews: {total_reviews - unique_review_ids:,}")
    
    if total_reviews != unique_review_ids:
        # Find duplicates
        duplicates = df[df['review_id'].duplicated(keep=False)]
        duplicate_ids = duplicates['review_id'].unique()
        
        print(f"\nNumber of review IDs with duplicates: {len(duplicate_ids)}")
        print("\nSample duplicate review IDs:")
        for review_id in duplicate_ids[:5]:  # Show first 5
            dup_records = df[df['review_id'] == review_id]
            print(f"\nReview ID: {review_id}")
            print(f"Appears {len(dup_records)} times")
            print(dup_records[['review_id', 'order_id', 'review_score', 'review_creation_date', 'review_comment_message']].to_string(index=False))
    
    return total_reviews - unique_review_ids

if __name__ == "__main__":
    duplicates = check_review_duplicates()

=== REVIEW DUPLICATES ANALYSIS ===
Total reviews: 99,224
Unique review IDs: 98,410
Duplicate reviews: 814

Number of review IDs with duplicates: 789

Sample duplicate review IDs:

Review ID: 28642ce6250b94cc72bc85960aec6c62
Appears 2 times
                       review_id                         order_id  review_score review_creation_date review_comment_message
28642ce6250b94cc72bc85960aec6c62 e239d280236cdd3c40cb2c033f681d1c             5  2018-03-25 00:00:00                    NaN
28642ce6250b94cc72bc85960aec6c62 bc42a955f289870d5789e6e437206300             5  2018-03-25 00:00:00                    NaN

Review ID: a0a641414ff718ca079b3967ef5c2495
Appears 2 times
                       review_id                         order_id  review_score review_creation_date review_comment_message
a0a641414ff718ca079b3967ef5c2495 169d7e0fd71d624d306f132acd791cbe             5  2018-03-04 00:00:00                    NaN
a0a641414ff718ca079b3967ef5c2495 4e93b736e8d687bca088c6ee496437e8             5

The duplicates show that 814 reviews have the same review_id but different order_ids. This suggests the same review was applied to multiple orders - possibly copy-paste reviews or system errors.
Looking at the pattern:

Same review_id, same review_score, same creation_date
Different order_ids
Often same review_comment_message

--> FIX LOAD REVIEWS --> sqlite3 olist_dashboard.db < sql/01_create_schema.sql 